In [121]:
from helius import NFTAPI, NameAPI, BalancesAPI, WebhooksAPI, TransactionsAPI
from dotenv import load_dotenv
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import os
import time
import json
import joblib

In [122]:
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_columns', None)   
pd.set_option('display.width', 200)    
pd.set_option('display.float_format', '{:.8f}'.format)

In [123]:
df = joblib.load('df_final_sorted.joblib')  # or .pkl
df

,epoch,starting_slot,slots_in_epoch,network,created_at,total_rewards,total_active_stake,timestamp,nodePubkey,commission,activatedStake,lastVote,rootSlot,epochVoteAccount,credits,previous_credits,credits_earned,votePubkey,activatedStake_SOL,activatedStake_USD,vote_account,total_reward_SOL,total_active_stake_SOL,name,keybase_id,www_url,details,avatar_url,details
0,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT,0,28931907684368,339613812,339613781,True,685159878,684174907,984971,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,28931.90768437,5000301.60508932,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,NaN,NaN,Nodes24.io 💎Jito 0% MEV💥,,https://nodes24.io/,Nodes24 is a reliable staking validator for your crypto,https://nodes24.io/images/apple-touch-icon.png,Nodes24 is a reliable staking validator for your crypto
1,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,FSyAsxcE7g8pSSEu5nx7Hkz44rMZiYio5Wz8Lszh3Nbi,0,198057028904298,339613812,339613781,True,665903209,664918221,984988,QXmsTYFK7YT2BpP2AnvXwuRpfwmsJZpovLcUqdSjoK1,198057.02890430,34230196.30552983,QXmsTYFK7YT2BpP2AnvXwuRpfwmsJZpovLcUqdSjoK1,NaN,NaN,Honeybee 🐝 +Jito,,None,"Support 24/7, high uptime, +MEV jito. BeeHappy :)",https://i.postimg.cc/FsrKYd56/bee.png,"Support 24/7, high uptime, +MEV jito. BeeHappy :)"
2,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,FWSHEWDt9ufmg3JtmWzhay5zncUiJHA7Xz9hBmdx2pCU,1,2071677019,339613812,339613781,True,544223539,543242774,980765,62vDttWmru9mXm1cFBcHa8f7ymya46gqfUTjyZd61M66,2.07167702,358.04793919,62vDttWmru9mXm1cFBcHa8f7ymya46gqfUTjyZd61M66,NaN,NaN,None,None,None,None,None,None
3,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,FVZLnRhj9Gjf77CtDFvh7jNu8avAnduJW5m365HrDLfD,0,52564812959086,339613812,339613781,True,589891722,588908389,983333,EijMpL5u45wg1ZTJZa4i7DvUsn8um8s5woVJsNmvCpTP,52564.81295909,9084776.62371883,EijMpL5u45wg1ZTJZa4i7DvUsn8um8s5woVJsNmvCpTP,NaN,NaN,Han SOLana + Wookiee MEV,,https://hansolana.fun/,"A long time ago, in a fast blockchain…\n\nHan SOLana and Wookiee MEV sign transactions so fast fees panic!\n\nDarkness approaches... A wicked validator with old hardware and hunger for sandwiches plans to raise gas fees!\n\nCan Han SOLana save the network, or w",https://i.pinimg.com/736x/78/84/da/7884da2c7b3813094f03e09ca581e334.jpg,"A long time ago, in a fast blockchain…\n\nHan SOLana and Wookiee MEV sign transactions so fast fees panic!\n\nDarkness approaches... A wicked validator with old hardware and hunger for sandwiches plans to raise gas fees!\n\nCan Han SOLana save the network, or w"
4,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,FUyx2W6wDt7u363QgQRWQYuytE5uJWZLGJpuVh3RDiCa,0,20944168946507,339613812,339613781,True,773968574,772983643,984931,AZUYzQX7nrbN1iE25YsSCSZAywZcyr1C1anMnwZtaqX9,20944.16894651,3619780.71902481,AZUYzQX7nrbN1iE25YsSCSZAywZcyr1C1anMnwZtaqX9,NaN,NaN,blockconsens.us,blockconsensus,https://blockconsens.us,,https://s3.amazonaws.com/keybase_processed_uploads/1594481f0102efd41478816dd60e5a05_360_360.jpg,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,769,332208000,432000,mainnet,2025-04-08T23:44:04.000Z,150140890931048.00000000,383083844124170560.00000000,2025-05-12 22:18:07.009603+00:00,9YN6VSNh5mzNGPaHTq13Y69Uu3Dc8sQEZgQFaS7YNi7d,0,999023721,338544474,338544443,True,1996119,272456,1723663,7yBAzmPzK4oNWs6PTnrSGJzGR4kYGDC2323nhuyKUZ6B,0.99902372,172.66126970,7yBAzmPzK4oNWs6PTnrSGJzGR4kYGDC2323nhuyKUZ6B,150140.89093105,383083844.12417054,NaN,NaN,NaN,NaN,NaN,NaN
6596,766,330912000,432000,mainnet,2025-04-03T00:04:01.000Z,151035887828837.00000000,384502142003974336.00000000,2025-05-12 22:18:07.009603+00:00,Syrd2b19zpvDrSTSkUzRUoPm9xpJkPtfKj77oGDCUHN,0,3220623

In [124]:
df = df.dropna(subset=['name']).reset_index(drop=True)

df.head()

,epoch,starting_slot,slots_in_epoch,network,created_at,total_rewards,total_active_stake,timestamp,nodePubkey,commission,activatedStake,lastVote,rootSlot,epochVoteAccount,credits,previous_credits,credits_earned,votePubkey,activatedStake_SOL,activatedStake_USD,vote_account,total_reward_SOL,total_active_stake_SOL,name,keybase_id,www_url,details,avatar_url,details
0,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT,0,28931907684368,339613812,339613781,True,685159878,684174907,984971,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,28931.90768437,5000301.60508932,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,NaN,NaN,Nodes24.io 💎Jito 0% MEV💥,,https://nodes24.io/,Nodes24 is a reliable staking validator for your crypto,https://nodes24.io/images/apple-touch-icon.png,Nodes24 is a reliable staking validator for your crypto
1,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,FSyAsxcE7g8pSSEu5nx7Hkz44rMZiYio5Wz8Lszh3Nbi,0,198057028904298,339613812,339613781,True,665903209,664918221,984988,QXmsTYFK7YT2BpP2AnvXwuRpfwmsJZpovLcUqdSjoK1,198057.02890430,34230196.30552983,QXmsTYFK7YT2BpP2AnvXwuRpfwmsJZpovLcUqdSjoK1,NaN,NaN,Honeybee 🐝 +Jito,,None,"Support 24/7, high uptime, +MEV jito. BeeHappy :)",https://i.postimg.cc/FsrKYd56/bee.png,"Support 24/7, high uptime, +MEV jito. BeeHappy :)"
2,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,FVZLnRhj9Gjf77CtDFvh7jNu8avAnduJW5m365HrDLfD,0,52564812959086,339613812,339613781,True,589891722,588908389,983333,EijMpL5u45wg1ZTJZa4i7DvUsn8um8s5woVJsNmvCpTP,52564.81295909,9084776.62371883,EijMpL5u45wg1ZTJZa4i7DvUsn8um8s5woVJsNmvCpTP,NaN,NaN,Han SOLana + Wookiee MEV,,https://hansolana.fun/,"A long time ago, in a fast blockchain…\n\nHan SOLana and Wookiee MEV sign transactions so fast fees panic!\n\nDarkness approaches... A wicked validator with old hardware and hunger for sandwiches plans to raise gas fees!\n\nCan Han SOLana save the network, or w",https://i.pinimg.com/736x/78/84/da/7884da2c7b3813094f03e09ca581e334.jpg,"A long time ago, in a fast blockchain…\n\nHan SOLana and Wookiee MEV sign transactions so fast fees panic!\n\nDarkness approaches... A wicked validator with old hardware and hunger for sandwiches plans to raise gas fees!\n\nCan Han SOLana save the network, or w"
3,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,FUyx2W6wDt7u363QgQRWQYuytE5uJWZLGJpuVh3RDiCa,0,20944168946507,339613812,339613781,True,773968574,772983643,984931,AZUYzQX7nrbN1iE25YsSCSZAywZcyr1C1anMnwZtaqX9,20944.16894651,3619780.71902481,AZUYzQX7nrbN1iE25YsSCSZAywZcyr1C1anMnwZtaqX9,NaN,NaN,blockconsens.us,blockconsensus,https://blockconsens.us,,https://s3.amazonaws.com/keybase_processed_uploads/1594481f0102efd41478816dd60e5a05_360_360.jpg,
4,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-12 22:18:07.009603+00:00,FUURpC3LjVnxr21PmEfHtxT7Mfe4CVJXxESBjQPvmqTZ,0,289213698736012,339613812,339613781,True,633535139,632550167,984972,2w4dcnbJDcGrAh4CFAYYpAaEJiUB2q1rFMGByBuB5Cqz,289213.69873601,49984803.55254495,2w4dcnbJDcGrAh4CFAYYpAaEJiUB2q1rFMGByBuB5Cqz,NaN,NaN,Vlaber01,,https://discordapp.com/users/880538802479857694,Ukraine blockchain Solana,https://i.ibb.co/L0Hg3yK/ecosystem-solana.png,Ukraine blockchain Solana


In [ ]:
latest_epoch = df['epoch'].max()

# Filter out the rows with the latest epoch where total_rewards and total_active_stake are NaN
df_cleaned = df[~((df['epoch'] == latest_epoch) & df['total_rewards'].isna() & df['total_active_stake'].isna())].reset_index(drop=True)

df_cleaned.head(n=5)



,epoch,starting_slot,slots_in_epoch,network,created_at,total_rewards,total_active_stake,timestamp,nodePubkey,commission,activatedStake,lastVote,rootSlot,epochVoteAccount,credits,previous_credits,credits_earned,votePubkey,activatedStake_SOL,activatedStake_USD,vote_account,total_reward_SOL,total_active_stake_SOL,name,keybase_id,www_url,details,avatar_url,details
0,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FLWc77X8dKh5RdJe5xMFxry8kvSVUbo9G4MQ8hCAg5ve,0,94355116478825,339613812,339613781,True,759218373,752317629,6900744,EcZBjDPzSZLdsyKCBq26MZMYFfiinZsfcL3SwWMK1eNL,94355.11647883,16307394.78103533,EcZBjDPzSZLdsyKCBq26MZMYFfiinZsfcL3SwWMK1eNL,149125.95670135,390905360.20912164,WALL·E,,https://walle.co.za,"I dove into Solana's universe, mastering every aspect.",https://files.catbox.moe/6au3zf.png,"I dove into Solana's universe, mastering every aspect."
1,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FU8F2V8yCFhseHDc1CJ5Sj1e5AbGNh3MnSe4REYX7a3P,0,5445170265861,339613812,339613781,True,663862661,656966327,6896334,A9YmK82uhCXieM9Cw9gN7W1F2KCxa8U1ArA6qxnyYcXQ,5445.17026586,941088.77704876,A9YmK82uhCXieM9Cw9gN7W1F2KCxa8U1ArA6qxnyYcXQ,149125.95670135,390905360.20912164,C-Key Validators | 0% fee + MEV,,https://bit.ly/4d7hUv0,Reliable staking. High result.,https://i.imgur.com/IbEWOmT.jpeg,Reliable staking. High result.
2,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FUURpC3LjVnxr21PmEfHtxT7Mfe4CVJXxESBjQPvmqTZ,0,289213698736012,339613812,339613781,True,632550167,625649395,6900772,2w4dcnbJDcGrAh4CFAYYpAaEJiUB2q1rFMGByBuB5Cqz,289213.69873601,49984803.55254495,2w4dcnbJDcGrAh4CFAYYpAaEJiUB2q1rFMGByBuB5Cqz,149125.95670135,390905360.20912164,Vlaber01,,https://discordapp.com/users/880538802479857694,Ukraine blockchain Solana,https://i.ibb.co/L0Hg3yK/ecosystem-solana.png,Ukraine blockchain Solana
3,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FSyAsxcE7g8pSSEu5nx7Hkz44rMZiYio5Wz8Lszh3Nbi,0,198057028904298,339613812,339613781,True,664918221,658017497,6900724,QXmsTYFK7YT2BpP2AnvXwuRpfwmsJZpovLcUqdSjoK1,198057.02890430,34230196.30552983,QXmsTYFK7YT2BpP2AnvXwuRpfwmsJZpovLcUqdSjoK1,149125.95670135,390905360.20912164,Honeybee 🐝 +Jito,,None,"Support 24/7, high uptime, +MEV jito. BeeHappy :)",https://i.postimg.cc/FsrKYd56/bee.png,"Support 24/7, high uptime, +MEV jito. BeeHappy :)"
4,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FPwKd8WmeugZLZySCcr5RmqdVnQvJ5zRQRCy6tnkdLQF,0,92072404267378,339613812,339613781,True,673802024,666901283,6900741,7okgo2YWz8fgNLAHSmAFwqPF59GMq9QuTSFSzPkkVcq7,92072.40426738,15912873.62953094,7okgo2YWz8fgNLAHSmAFwqPF59GMq9QuTSFSzPkkVcq7,149125.95670135,390905360.20912164,Tokyo ㊗️ 0%fee+Jito,,None,"Stake with Tokyo and benefit from 0% commission, plus additional earnings through MEV. Maximize your staking returns with Tokyo. High security and 99.9% uptime",https://i.postimg.cc/T2n7kDRn/tokyo.png,"Stake with Tokyo and benefit from 0% commission, plus additional earnings through MEV. Maximize your staking returns with Tokyo. High security and 99.9% uptime"


In [129]:
validator_performance = df_cleaned[
    ['avatar_url', 'name', 'activatedStake_SOL', 'commission', 
     'credits_earned', 'details']
    ]

validator_performance

,avatar_url,name,activatedStake_SOL,commission,credits_earned,details,details
0,https://files.catbox.moe/6au3zf.png,WALL·E,94355.11647883,0,6900744,"I dove into Solana's universe, mastering every aspect.","I dove into Solana's universe, mastering every aspect."
1,https://i.imgur.com/IbEWOmT.jpeg,C-Key Validators | 0% fee + MEV,5445.17026586,0,6896334,Reliable staking. High result.,Reliable staking. High result.
2,https://i.ibb.co/L0Hg3yK/ecosystem-solana.png,Vlaber01,289213.69873601,0,6900772,Ukraine blockchain Solana,Ukraine blockchain Solana
3,https://i.postimg.cc/FsrKYd56/bee.png,Honeybee 🐝 +Jito,198057.02890430,0,6900724,"Support 24/7, high uptime, +MEV jito. BeeHappy :)","Support 24/7, high uptime, +MEV jito. BeeHappy :)"
4,https://i.postimg.cc/T2n7kDRn/tokyo.png,Tokyo ㊗️ 0%fee+Jito,92072.40426738,0,6900741,"Stake with Tokyo and benefit from 0% commission, plus additional earnings through MEV. Maximize your staking returns with Tokyo. High security and 99.9% uptime","Stake with Tokyo and benefit from 0% commission, plus additional earnings through MEV. Maximize your staking returns with Tokyo. High security and 99.9% uptime"
...,...,...,...,...,...,...,...
4140,https://drive.google.com/file/d/1kzh_nrh_LjOnr5B95AVY64tdOwsIC8Cx/view,Green Labs | Nordic Waves,5306.34966923,100,6896981,"0% commission, MEV rewards + professional high-performance validator + validator powered by green energy + research on green tech for blockchain. Discord username: green.labs","0% commission, MEV rewards + professional high-performance validator + validator powered by green energy + research on green tech for blockchain. Discord username: green.labs"
4141,https://s3.amazonaws.com/keybase_processed_uploads/cb34ff2c547805b4572d2d316b2c7b05_360_360.jpg,0xDEADNODE,3720.60835797,0,6896986,,
4142,https://i.postimg.cc/pTQd06pv/x.jpg,DeezNode XYZ,1302.42462429,0,5656818,"if you know, you know.","if you know, you know."
4143,https://programmist1s.ru/wp-content/uploads/2024/01/txc.jpg,toxicoff,103.98794650,5,6887844,,


#### total_reward_SOL represents the network-wide rewards for the epoch, not the rewards earned by a specific validator. Dividing total_reward_SOL by activatedStake_SOL doesn’t give the validator’s reward rate. Instead, you need to calculate the validator’s share of total_reward_SOL based on their active_stake relative to total_active_stake.



Here's the breakdown:
Step 1: Validator's Share of Total Rewards
The formula to compute the validator's rewards is:

Validator Rewards (SOL) = (Validator Active Stake /Total Active Stake) × Total Rewards (SOL)



In [ ]:
df_cleaned['validator_rewards_SOL'] = (df_cleaned['activatedStake_SOL'] / df_cleaned['total_active_stake_SOL']) * df_cleaned['total_reward_SOL']
df_cleaned.head()




,epoch,starting_slot,slots_in_epoch,network,created_at,total_rewards,total_active_stake,timestamp,nodePubkey,commission,activatedStake,lastVote,rootSlot,epochVoteAccount,credits,previous_credits,credits_earned,votePubkey,activatedStake_SOL,activatedStake_USD,vote_account,total_reward_SOL,total_active_stake_SOL,name,keybase_id,www_url,details,avatar_url,details,validator_rewards_SOL
0,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FLWc77X8dKh5RdJe5xMFxry8kvSVUbo9G4MQ8hCAg5ve,0,94355116478825,339613812,339613781,True,759218373,752317629,6900744,EcZBjDPzSZLdsyKCBq26MZMYFfiinZsfcL3SwWMK1eNL,94355.11647883,16307394.78103533,EcZBjDPzSZLdsyKCBq26MZMYFfiinZsfcL3SwWMK1eNL,149125.95670135,390905360.20912164,WALL·E,,https://walle.co.za,"I dove into Solana's universe, mastering every aspect.",https://files.catbox.moe/6au3zf.png,"I dove into Solana's universe, mastering every aspect.",35.99540566
1,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FU8F2V8yCFhseHDc1CJ5Sj1e5AbGNh3MnSe4REYX7a3P,0,5445170265861,339613812,339613781,True,663862661,656966327,6896334,A9YmK82uhCXieM9Cw9gN7W1F2KCxa8U1ArA6qxnyYcXQ,5445.17026586,941088.77704876,A9YmK82uhCXieM9Cw9gN7W1F2KCxa8U1ArA6qxnyYcXQ,149125.95670135,390905360.20912164,C-Key Validators | 0% fee + MEV,,https://bit.ly/4d7hUv0,Reliable staking. High result.,https://i.imgur.com/IbEWOmT.jpeg,Reliable staking. High result.,2.07727063
2,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FUURpC3LjVnxr21PmEfHtxT7Mfe4CVJXxESBjQPvmqTZ,0,289213698736012,339613812,339613781,True,632550167,625649395,6900772,2w4dcnbJDcGrAh4CFAYYpAaEJiUB2q1rFMGByBuB5Cqz,289213.69873601,49984803.55254495,2w4dcnbJDcGrAh4CFAYYpAaEJiUB2q1rFMGByBuB5Cqz,149125.95670135,390905360.20912164,Vlaber01,,https://discordapp.com/users/880538802479857694,Ukraine blockchain Solana,https://i.ibb.co/L0Hg3yK/ecosystem-solana.png,Ukraine blockchain Solana,110.33174243
3,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FSyAsxcE7g8pSSEu5nx7Hkz44rMZiYio5Wz8Lszh3Nbi,0,198057028904298,339613812,339613781,True,664918221,658017497,6900724,QXmsTYFK7YT2BpP2AnvXwuRpfwmsJZpovLcUqdSjoK1,198057.02890430,34230196.30552983,QXmsTYFK7YT2BpP2AnvXwuRpfwmsJZpovLcUqdSjoK1,149125.95670135,390905360.20912164,Honeybee 🐝 +Jito,,None,"Support 24/7, high uptime, +MEV jito. BeeHappy :)",https://i.postimg.cc/FsrKYd56/bee.png,"Support 24/7, high uptime, +MEV jito. BeeHappy :)",75.55650785
4,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000,2025-05-12 22:18:07.009603+00:00,FPwKd8WmeugZLZySCcr5RmqdVnQvJ5zRQRCy6tnkdLQF,0,92072404267378,339613812,339613781,True,673802024,666901283,6900741,7okgo2YWz8fgNLAHSmAFwqPF59GMq9QuTSFSzPkkVcq7,92072.40426738,15912873.62953094,7okgo2YWz8fgNLAHSmAFwqPF59GMq9QuTSFSzPkkVcq7,149125.95670135,390905360.20912164,Tokyo ㊗️ 0%fee+Jito,,None,"Stake with Tokyo and benefit from 0% commission, plus additional earnings through MEV. Maximize your staking returns with Tokyo. High security and 99.9% uptime",https://i.postimg.cc/T2n7kDRn/tokyo.png,"Stake with Tokyo and benefit from 0% commission, plus additional earnings through MEV. Maximize your staking returns with Tokyo. High security and 99.9% uptime",35.12457687


In [ ]:
# df_cleaned.dtypes

In [ ]:
df_cleaned['Uptime'] = (df_cleaned['credits_earned'] / df_cleaned['slots_in_epoch']) * 100
df_cleaned['Uptime'] = df_cleaned['Uptime'].clip(upper=100)  # Cap at 100% due to high credits_earned


#### Epochs per Year (182.5):
The assumption of 182.5 epochs per year is slightly off. A Solana epoch is ~2.43 days (432,000 slots × 400ms per slot ≈ 172,800 seconds ≈ 2.43 days). Thus:

Epochs per Year = 365 / 2.43 ≈ 150.21



In [ ]:
# EPOCHS_PER_YEAR = 365 / 2.43  # ≈150.21

# row = df_cleaned.loc[1]  # row 2 (index 1)
# reward = row["validator_rewards_SOL"]
# stake = row["activatedStake_SOL"]

# apy = ((1 + reward / stake) ** EPOCHS_PER_YEAR - 1) * 100
# print(apy)


In [ ]:
EPOCHS_PER_YEAR = 365 / 2.43  # ~150.21 epochs per year
df_cleaned["apy"] = ((1 + df_cleaned["validator_rewards_SOL"] / df_cleaned['activatedStake_SOL']) ** EPOCHS_PER_YEAR  -1) * 100
df_cleaned["apy"]


0      5.89637815
1      5.89637815
2      5.89637815
3      5.89637815
4      5.89637815
          ...    
4140   5.91095400
4141   5.91095400
4142   5.91095400
4143   5.91095400
4144   5.91095400
Name: apy, Length: 4145, dtype: float64

In [ ]:
# df_cleaned = df_cleaned.sort_values(['nodePubkey', 'epoch'])
# df_cleaned['Prev_ActivatedStake'] = df_cleaned.groupby('nodePubkey')['activatedStake_SOL'].shift(1)
# df_cleaned['Stake_Inflow_Percent'] = ((df_cleaned['activatedStake_SOL'] - df_cleaned['Prev_ActivatedStake']) / df_cleaned['Prev_ActivatedStake']) * 100
# # If no previous epoch, set to NaN
# df_cleaned['Stake_Inflow_Percent'] = df_cleaned['Stake_Inflow_Percent'].fillna(0)  # or np.nan if preferred

# df_cleaned['Stake_Inflow_Percent']


In [ ]:
# EPOCHS_PER_YEAR = 365 / 2.43  # ~150.21 epochs/year

# # Calculate daily return equivalent first
# df_cleaned['daily_return'] = df_cleaned['validator_rewards_SOL'] / df_cleaned['activatedStake_SOL'] * (86400/400)  # Slots/day

# # Calculate annualized APY using geometric mean
# def calculate_apy(group):
#     if group['activatedStake_SOL'].sum() == 0:
#         return 0
#     return (np.expm1(np.log1p(group['daily_return']).mean() * 365) * 100)

# validator_apy = df_cleaned.groupby('votePubkey').apply(calculate_apy)
# validator_apy

In [ ]:
# EPOCHS_PER_YEAR = 365 / 2.43  # ~150.21 epochs per year
# # df_cleaned['apy'] = (
# #     (1 + df_cleaned['validator_rewards_SOL'] / df_cleaned['activatedStake_SOL'])**EPOCHS_PER_YEAR - 1) * 100  # Convert to percentage

# # # Group by votePubkey and compute mean APY per epoch, then annualize
# # validator_apy = df_cleaned.groupby('votePubkey')['apy'].mean() * 365 / df_cleaned['slots_in_epoch'].mean()



# df_cleaned['apy'] = (
#     (1 + df_cleaned['validator_rewards_SOL'] / df_cleaned['activatedStake_SOL'])**EPOCHS_PER_YEAR - 1) * 100


# df_cleaned.head()

In [ ]:
# apy = ((1 + df_cleaned['validator_rewards_SOL'] / df_cleaned['activatedStake_SOL']) ** 150.21 -1) * 100
# apy